In [1]:
from pyspark.sql import SparkSession

# Spark session & context
spark = SparkSession.builder.master("spark://spark:7077").appName("jupyter-notebook").config("spark.driver.memory", "512m").getOrCreate()
sc = spark.sparkContext

# Sum of the first 100 whole numbers
rdd = sc.parallelize(range(1000+1))
rdd.sum()
print("hello")
# 5050

hello


# WOW
Nose


In [5]:
spark.stop()
print("wow")